In [ ]:
from typing import List, Tuple
from pyhepmc import GenEvent, GenParticle, ReaderAsciiHepMC2, FourVector


def get_z_decay_electrons(event):
    electrons = []
    for p in event.particles:
        if p.pid == 23:
            vz = p.end_vertex
            if vz is None:
                continue
            for dau in vz.particles_out:
                if dau.status == 1 and abs(dau.pid) == 11:
                    electrons.append(dau)
    return electrons


def extract_electron_momenta(event):
    electrons = get_z_decay_electrons(event)
    if len(electrons) < 2:
        raise ValueError(f"Expected at least 2 final-state electrons/positrons, found {len(electrons)}")
    e1, e2 = electrons[0], electrons[1]
    return (e1.momentum, e2.momentum)


def extract_electron_momenta_from_hepmc2(input_path):
    momenta_list = []
    total = 0
    skipped = 0

    with ReaderAsciiHepMC2(input_path) as reader:
        while not reader.failed():
            evt = GenEvent()
            reader.read_event(evt)
            if reader.failed():
                break

            total += 1
            try:
                mom_pair = extract_electron_momenta(evt)
                momenta_list.append(mom_pair)
            except ValueError:
                skipped += 1
    print(f"Processed {total} events, extracted from {len(momenta_list)} events, skipped {skipped} events.")
    return momenta_list


def write_momenta_to_csv(momenta,output_path):
    header = ['px1', 'py1', 'pz1', 'e1', 'px2', 'py2', 'pz2', 'e2']
    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        for (p1, p2) in momenta:
            writer.writerow([p1.px, p1.py, p1.pz, p1.e,
                             p2.px, p2.py, p2.pz, p2.e])
